# Deep Convolutional Generative Adversarial Network



### Loading datset and preprocessing

In [ ]:
from tensorflow.keras.datasets import mnist

# load mnist x_train
(x_train, _), (_, _) = mnist.load_data()

In [ ]:
import numpy as np

""" Preprocessing """

# create batch_dim
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)

# convert to keras compatible float
x_train = x_train.astype(np.float32)

# normalization to inputs between [-1, 1]
MAX = np.max(x_train)
x_train = (x_train / (MAX/2)) - 1

print(f"Data_shape: {x_train.shape} - Input_range: [{np.min(x_train)}, {np.max(x_train)}]")

Data_shape: (60000, 28, 28, 1) - Input_range: [-1.0, 1.0]


### Discriminator Network


In [ ]:
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

img_shape = x_train.shape[1:] # (28, 28, 1)

DISCRIMINATOR = Sequential()
DISCRIMINATOR.add(Conv2D(filters=64, kernel_size=5, strides=2, padding="same", input_shape=img_shape))
DISCRIMINATOR.add(LeakyReLU(alpha=0.2))
DISCRIMINATOR.add(Dropout(rate=0.3))
DISCRIMINATOR.add(Conv2D(filters=128, kernel_size=5, strides=2, padding="same"))
DISCRIMINATOR.add(LeakyReLU(alpha=0.2))
DISCRIMINATOR.add(Dropout(rate=0.3))
DISCRIMINATOR.add(Flatten())
DISCRIMINATOR.add(Dense(1))
DISCRIMINATOR.add(Activation("sigmoid")) # sigmoid range [0,1] 0 - fake IMG, 1 real IMG

d_input = Input(shape=img_shape) # image in
d_pred = DISCRIMINATOR(d_input) # prediction in % out

DISCRIMINATOR = Model(inputs=d_input, outputs=d_pred)
DISCRIMINATOR.compile(
    loss="binary_crossentropy",
    optimizer=Adam(learning_rate=0.0002, beta_1=0.5),
    metrics=["accuracy"]
)

### Generator Net

In [ ]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import UpSampling2D

z_dim = 100 # Noise Vector size

GENERATOR = Sequential()
GENERATOR.add(Dense(units=7 * 7 * 128, input_dim=z_dim)) # 7x7 IMG -> upsacling twice to 28x28 (128 is kernel_dim)
GENERATOR.add(LeakyReLU(alpha=0.2))
GENERATOR.add(Reshape(target_shape=(7, 7, 128)))
GENERATOR.add(UpSampling2D()) # 14x14x128
GENERATOR.add(Conv2D(filters=128, kernel_size=5, strides=1, padding="same", use_bias=False))
GENERATOR.add(BatchNormalization())
GENERATOR.add(LeakyReLU(alpha=0.2))
GENERATOR.add(UpSampling2D())
GENERATOR.add(Conv2D(filters=64, kernel_size=5, strides=1, padding="same", use_bias=False))
GENERATOR.add(BatchNormalization())
GENERATOR.add(LeakyReLU(alpha=0.2))
GENERATOR.add(Conv2D(filters=img_shape[-1], kernel_size=5, strides=1, padding="same", use_bias=False))
GENERATOR.add(Activation("tanh"))

noise = Input(shape=(z_dim,)) # noise in
img = GENERATOR(noise) # generated image out

GENERATOR = Model(inputs=noise, outputs=img)
# GENERATOR isn't compiled, because of GAN Net

### DCGAN Net

In [ ]:
# combining DISCRIMINATOR and GENERATOR to GAN
noise_in = Input(shape=(z_dim,)) # noise in
img = GENERATOR(noise_in) # generated image
DISCRIMINATOR.trainable = False
d_pred = DISCRIMINATOR(img) # prediction if image input is real or fake
DCGAN = Model(inputs=noise_in, outputs=d_pred)

DCGAN.compile(
    loss="binary_crossentropy",
    optimizer=Adam(learning_rate=0.0001)
)

### Training

In [ ]:
import matplotlib.pyplot as plt
import os

PATH = "generated_images"
try:
    os.makedirs(PATH)
except FileExistsError:
    pass

# creating labels for dataset
batch_size = 32
y_real = np.ones(shape=(batch_size, 1)) # 1 = 100% real
y_fake = np.zeros(shape=(batch_size, 1)) # 0 = 0% real

epochs=20_000

for epoch in range(1, epochs+1):
    # get random images from real dataset
    rand_idxs = np.random.randint(low=0, high=x_train.shape[0], size=batch_size)
    real_imgs = x_train[rand_idxs]
    # generate fake images for training
    noise = np.random.normal(loc=0.0, scale=1.0, size=(batch_size, z_dim)) # noise vector (32, 100)
    fake_imgs = GENERATOR(noise, training=False)
    # train DISCRIMINATOR
    d_loss_real = DISCRIMINATOR.train_on_batch(real_imgs, y_real)
    d_loss_fake = DISCRIMINATOR.train_on_batch(fake_imgs, y_fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    # train GENERATOR
    noise = np.random.normal(loc=0.0, scale=1.0, size=(batch_size, z_dim))
    g_loss = DCGAN.train_on_batch(noise, y_real)

    if(epoch % 100)==0:
        print(
            f"{epoch} - D_loss: {round(d_loss[0], 4)}"
            f" D_acc: {round(d_loss[1], 4)}"
            f" G_loss: {round(g_loss, 4)}"
        )

    # saving every 1000 steps
    if(epoch % 1000) == 0:
        print("SAVED")

        rows, cols = 5, 5
        gen_imgs = GENERATOR.predict(noise) # generating image
        gen_imgs = 0.5 * gen_imgs + 0.5
        fig, axs = plt.subplots(rows, cols)
        cnt = 0
        for i in range(rows):
            for j in range(cols):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap="gray")
                axs[i, j].axis("off")
                cnt +=1
        img_name = f"{epoch}.png"
        fig.savefig(os.path.join(PATH, img_name))
        plt.close()

100 - D_loss: 0.5201 D_acc: 0.8125 G_loss: 1.0727
200 - D_loss: 0.592 D_acc: 0.7188 G_loss: 0.9452
300 - D_loss: 0.684 D_acc: 0.5469 G_loss: 1.0026
400 - D_loss: 0.6206 D_acc: 0.6562 G_loss: 0.8844
500 - D_loss: 0.6684 D_acc: 0.5156 G_loss: 0.8493
600 - D_loss: 0.5861 D_acc: 0.6562 G_loss: 1.0913
700 - D_loss: 0.6401 D_acc: 0.6875 G_loss: 0.8195
800 - D_loss: 0.6406 D_acc: 0.625 G_loss: 0.8371
900 - D_loss: 0.6162 D_acc: 0.6406 G_loss: 0.8277
1000 - D_loss: 0.6849 D_acc: 0.5938 G_loss: 0.8145
SAVED
1100 - D_loss: 0.6426 D_acc: 0.6094 G_loss: 0.9236
1200 - D_loss: 0.5965 D_acc: 0.7188 G_loss: 0.8803
1300 - D_loss: 0.6222 D_acc: 0.6562 G_loss: 1.0084
1400 - D_loss: 0.7089 D_acc: 0.5469 G_loss: 0.7986
1500 - D_loss: 0.689 D_acc: 0.5312 G_loss: 0.8259
1600 - D_loss: 0.6613 D_acc: 0.6094 G_loss: 0.7718
1700 - D_loss: 0.6504 D_acc: 0.6406 G_loss: 0.848
1800 - D_loss: 0.6218 D_acc: 0.6719 G_loss: 0.8394
1900 - D_loss: 0.5926 D_acc: 0.7188 G_loss: 0.8434
2000 - D_loss: 0.6883 D_acc: 0.5469 G_l